In [1]:
from transformers import pipeline
import pandas as pd
from tqdm import tqdm
import torch

C:\Users\Nursulu_1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [37]:
import pickle

In [23]:
from collections import Counter

In [2]:
pipe = pipeline("zero-shot-classification", model="google/flan-t5-xxl")     

Loading checkpoint shards: 100%|██████████| 5/5 [00:02<00:00,  1.77it/s]
Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at google/flan-t5-xxl and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


In [2]:
# Free unused memory
torch.cuda.empty_cache()

# Optional: Check memory usage after clearing
print(torch.cuda.memory_allocated())
print(torch.cuda.memory_reserved())

0
0


In [3]:
pipe = pipeline("text2text-generation", model="google/flan-t5-xxl")     

Loading checkpoint shards: 100%|██████████| 5/5 [00:00<00:00,  6.53it/s]
Device set to use cuda:0


In [5]:
res = pipe(["Your task is to detect relationship between Text 1 and Text 2. It can be contradiction, entailment, or unrelated. Text 1: This restaurant is bad. Text 2: This restaurant is good. Relationship between Text 1 and Text 2:"])

In [6]:
res

[{'generated_text': 'contradiction'}]

In [7]:
all_samples_df = pd.read_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\all_samples_df.csv")

In [8]:
all_samples_df.columns = ["Input"]
all_samples_df['t5-xxl'] = 0
all_samples_df['confidence'] = 0

In [15]:
input = "Text 1: I hate apples. Text 2: I love apples."

In [16]:
# input = all_samples_df['Input'][0]
res = pipe([f"Your task is to detect relationship between Text 1 and Text 2. It can be contradiction, entailment, or unrelated.\n{input}\nRelationship between Text 1 and Text 2:"])
print(res[0]['generated_text'])

contradiction


In [17]:
rte_to_labels = {'contradiction': 'Inconsistent', 'entailment': 'Consistent', 'unrelated': 'Unrelated'}

In [18]:
answers_t5_xxl = []
for i in tqdm(range(len(all_samples_df))):
    input = all_samples_df['Input'][i]
    res = pipe([f"Your task is to detect relationship between Text 1 and Text 2. It can be contradiction, entailment, or unrelated.\n{input}\nRelationship between Text 1 and Text 2:"])
    answers_t5_xxl.append(rte_to_labels[res[0]['generated_text']])


 47%|████▋     | 331/700 [17:56<19:59,  3.25s/it]


KeyError: 'unanswerable'

In [21]:
i

331

In [22]:
for el in tqdm(range(i, len(all_samples_df))):
    input = all_samples_df['Input'][el]
    try:
        res = pipe([f"Your task is to detect relationship between Text 1 and Text 2. It can be contradiction, entailment, or unrelated.\n{input}\nRelationship between Text 1 and Text 2:"])
        answers_t5_xxl.append(rte_to_labels[res[0]['generated_text']])
    except:
        answers_t5_xxl.append('error')

100%|██████████| 369/369 [20:33<00:00,  3.34s/it]


In [25]:
t5_to_answer = dict(enumerate(answers_t5_xxl))

In [34]:
input = all_samples_df['Input'][334]
res = pipe([f"Your task is to detect relationship between Text 1 and Text 2. It can be contradiction, entailment, or unrelated.\n{input}\nRelationship between Text 1 and Text 2:"])


In [35]:
res

[{'generated_text': 'unanswerable'}]

In [36]:
answers_t5_xxl[334] = 'Unrelated'

331, 334 - said unanswerable. some sensitive topics like drugs and gender etc. we will count as unrelated

In [29]:
res

[{'generated_text': 'unanswerable'}]

In [ ]:
331
334

In [24]:
Counter(answers_t5_xxl)

Counter({'Unrelated': 535, 'Inconsistent': 145, 'Consistent': 18, 'error': 2})

In [38]:
# with open('answers_t5_xxl_3_types.pkl', 'wb') as file:
#     pickle.dump(answers_t5_xxl, file)

In [8]:
res = pipe(["Text 1: This restaurant is bad. Text 2: This restaurant is good. Relationship between Text 1 and Text 2:"], candidate_labels=["unrelated", "contradiction", "entailment"])

In [9]:
res

[{'sequence': 'Text 1: This restaurant is bad. Text 2: This restaurant is good. Relationship between Text 1 and Text 2:',
  'labels': ['contradiction', 'unrelated', 'entailment'],
  'scores': [0.3427811563014984, 0.33414897322654724, 0.32306990027427673]}]

In [12]:
res[0]['labels'][0]

'contradictory'

In [39]:
df = pd.read_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\results\df_gold_part_1.csv")

In [44]:
df

,Input,Answer,Confidence,All answers,Answer_3_classes,ChatGPT-4 5 classes,ChatGPT-4 3 classes,ChatGPT-4 5 classes aggregated,ChatGPT-3.5 5 classes,ChatGPT-3.5 3 classes,ChatGPT-3.5 5 classes aggregated,Llama3.3-70B-instruct 5 classes,Llama3.3-70B-instruct 3 classes,Llama3.3-70B-instruct 5 classes aggregated
0,Text 1: We will abolish long-term tuition fees...,Surface contradiction,100.0,"['Surface contradiction', 'Surface contradicti...",Inconsistent,Surface contradiction,Surface contradiction,Inconsistent,Surface contradiction,Inconsistent,Inconsistent,Surface contradiction,Inconsistent,Inconsistent
1,Text 1: The term of a prime minister should be...,Surface contradiction,40.0,"['Consistent', 'Surface contradiction', 'Indir...",Inconsistent,Consistent,Consistent,Consistent,Surface contradiction,Inconsistent,Inconsistent,Consistent,Consistent,Consistent
2,Text 1: Food that is free from genetic modific...,Consistent,40.0,"['Consistent', 'Consistent', 'Indirect inconsi...",Consistent,Consistent,Consistent,Consistent,Indirect inconsistency,Consistent,Inconsistent,Consistent,Consistent,Consistent
3,Text 1: The 5% hurdle prevents the representat...,Surface contradiction,60.0,"['Surface contradiction', 'Surface contradicti...",Inconsistent,Surface contradiction,Surface contradiction,Inconsistent,Surface contradiction,Inconsistent,Inconsistent,Surface contradiction,Inconsistent,Inconsistent
4,Text 1: Germany should consistently deport for...,Unrelated,100.0,"['Unrelated', 'Unrelated', 'Unrelated', 'Unrel...",Unrelated,Unrelated,Unrelated,Unrelated,Unrelated,Unrelated,Unrelated,Unrelated,"ChatCompletionMessage(content=""Label: Unrelate...",Unrelated
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,"Text 1: To bolster the economy, a major focus...",Indirect inconsistency,60.0,"['Indirect inconsistency', 'Indirect inconsist...",Inconsistent,Factual inconsistency,Indirect inconsistency,Inconsistent,Surface contradiction,Inconsistent,Inconsistent,Indirect inconsistency,Inconsistent,Inconsistent
344,Text 1: Students should have the free choice b...,Surface contradiction,60.0,"['Factual inconsistency', 'Factual inconsisten...",Inconsistent,Indirect inconsistency,Indirect inconsistency,Inconsistent,Surface contradiction,Inconsistent,Inconsistent,Indirect inconsistency,Consistent,Inconsistent
345,Text 1: Belonging to a certain social or ethni...,Indirect inconsistency,80.0,"['Indirect inconsistency', 'Indirect inconsist...",Inconsistent,Indirect inconsistency,Indirect inconsistency,Inconsistent,Surface contradiction,Inconsistent,Inconsistent,Consistent,Consistent,Consistent
346,Text 1: The level of social basic security mus...,Factual inconsistency,40.0,"['Indirect inconsistency', 'Factual inconsiste...",Inconsistent,Indirect inconsistency,Indirect inconsistency,Inconsistent,Surface contradiction,Inconsistent,Inconsistent,Indirect inconsistency,Inconsistent,Inconsistent


In [42]:
df_selected = df.iloc[:, [0, 1]]

In [45]:
df_selected_3_classes = df.iloc[:, [0, 4]]

In [46]:
df_selected.to_csv("fine_tuning_pt_1_5_classes.csv", index=False)

In [47]:
df_selected_3_classes.to_csv("fine_tuning_pt_1_3_classes.csv", index=False)